<ins>Logical Workflow PA Project</ins>

0. Call Data from SQL Server and save to local .pkl. Filter by TBM 1 and time span ('2022/02/01', '2022/04/01')
1. Join data for a complete dataset
2. Initial Filter (by Equipment ID: TBM 1 and remove Trial Recipes), key generation, cycle time generation and EDA
3. Left Merge Alarm mapping and Alarm hisroy on Created Key.
4. Alarm Discovery: Alarm to GT allocation: Calculate PAAP start and finish times if needed. Do NOT Filter alarms by Type e.g.  FF, CF and MC, assin name in#dexes. Link these alarm name indexes that are occurring for the same green tire at those times identified as PA. Create GT to Alarm Index Mapping.
5. One Hot encode Alarm Indexes before we carry out feature selction to see relevant alarms.
6. Feature selection: Initial Filter using statistical univariate filter methods. 
7. Feature selection: Narrow down using ML Models. Identify which steps are posing the issue (i.e. is it sidewall, inner liner, is it transport time etc.)
8. View Selected Alarms and Discuss with Operations Team

In [ ]:
conda install -c anaconda scipy==1.7.1
conda install -c anaconda pandas==1.3.4
conda install -c conda-forge statsmodels==0.13.1
conda install -c anaconda scikit-learn==0.23.2

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from matplotlib.pyplot import figure
import seaborn as sns

In [2]:
import matplotlib
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [3]:
alarm_list = [931, 529, 532, 113, 968, 220, 184, 999, 870, 692, 697, 269, 524]

In [4]:
ohe = pd.read_pickle("./pkl/alarm_OHE.pkl")
df_alarm_pa = pd.read_pickle("./pkl/alarms_join.pkl")

In [5]:
df_alarm_pa

,key,Name,Type,MessageID_x,HistoryMessageID,TimeStamp,Duration,EquipmentID,name_index,alarm_start
0,89031TBM_1,MOTION COMMUNICATION FAILURE CARCASS SIDE,FF,89031,21906293,2022-03-25 03:06:38.3988765,4,TBM 1,0,2022-03-25 03:06:34.398876500
1,89031TBM_1,MOTION COMMUNICATION FAILURE CARCASS SIDE,FF,89031,21337964,2022-02-27 11:32:38.1855400,4,TBM 1,0,2022-02-27 11:32:34.185540000
2,89031TBM_1,MOTION COMMUNICATION FAILURE CARCASS SIDE,FF,89031,21017480,2022-02-11 09:42:28.6704247,7,TBM 1,0,2022-02-11 09:42:21.670424700
3,89031TBM_1,MOTION COMMUNICATION FAILURE CARCASS SIDE,FF,89031,21018750,2022-02-11 11:10:13.6504815,4,TBM 1,0,2022-02-11 11:10:09.650481500
5,89031TBM_1,MOTION COMMUNICATION FAILURE CARCASS SIDE,FF,89031,22525429,2022-04-27 15:08:19.1819223,7,TBM 1,0,2022-04-27 15:08:12.181922300
...,...,...,...,...,...,...,...,...,...,...
1347317,97567TBM_1,???,OM,97567,22618603,2022-05-04 04:03:30.7745552,14,TBM 1,1018,2022-05-04 04:03:16.774555200
1347318,97567TBM_1,???,OM,97567,22495750,2022-04-25 21:30:33.4923961,23,TBM 1,1018,2022-04-25 21:30:10.492396100
1347319,97567TBM_1,???,OM,97567,22474746,2022-04-24 19:40:02.4974760,28,TBM 1,1018,2022-04-24 19:39:34.497476000
1347320,97567TBM_1,???,OM,97567,22474770,2022-04-24 19:41:43.7408270,58,TBM 1,1018,2022-04-24 19:40:45.740827000


In [6]:
alarm_map = df_alarm_pa[['Name', 'name_index', 'Type']].drop_duplicates()
alarm_map

,Name,name_index,Type
0,MOTION COMMUNICATION FAILURE CARCASS SIDE,0,FF
8,"FAILURE RIO 50, CARCASS SERVICER MAIN CABINET",1,FF
9,"FAILURE RIO 51, CARCASS SERVICER BODYPLY1",2,FF
14,"FAILURE RIO 52, CARCASS SERVICER INNERLINER",3,FF
15,"FAILURE RIO 59, CARCASS SERVICER BODYPLY2",4,FF
...,...,...,...
1340449,"BEADLOADER, LIGHT CURTAIN AT FRONT BLOCKED",1015,FF
1341484,PA MATERIAL STAYED ON CONVEYOR FOR TOO LONG,1016,OM
1341815,PA MATERIAL REJECTED DUE TO SHRINKAGE,1017,OM
1341888,???,1018,OM


In [8]:
alarm_list = [681, 1006] 
alarm_map[alarm_map['name_index'].isin(alarm_list)]

,Name,name_index,Type
702866,SAFETY SCREEN TBM SIDE OF CARCASS SIDE ACTIVE,681,OM
1283704,CC SIDE-AREA_ENTRY_OP_1_ACTIVE,1006,OM


In [8]:
alarm_map[alarm_map['name_index'].isin(alarm_list)]

,Name,name_index,Type
103842,"BREAKER 2, APPLY CYCLE BUSY",113,MC
156687,"CAPSTRIP LET-OFF, FESTOONER NOT AT OPERATING POS",184,MC
173903,SAFETY SCREEN (FLOOR) TBM B&T SIDE ACTIVE,220,OM
229084,"BT DRUM, MODULE DISABLED",269,FF
437371,"BP PRODUCTION, CYCLE BUSY",524,MC
441409,BP MATERIAL AT L.M. POS. DETECTOR,529,MC
516667,"BP STORAGE CONV, CYCLE BUSY",532,MC
770274,PREASSEMBLY LENGTH NOT CORRECT,692,OM
773322,PREASSEMBLY WIDTH NOT CORRECT,697,OM
863170,"CARCASS SIDE, SAFETY ACTIVATED",870,MC


572: They already knew about this one. Nothing can be done AI wise as this is material dependent, Have to remove sploce / material & will stop the material 
922: 
214: Due to BT preparation, PA will be slowed down. Not inherent to PA, just Belt/Tread side. As they both must meet in the middle transfer ring
852: Nterrupt the safety, the message 
883: MES message, not PA related 
637: They already knew about this one.
848: Robot - not related to PA
931: Recently modified in the quality stop cycle 
847: Robot - not related to PA 


931: ROBOT GRIPPER, MODULE DISABLED
529: BP MATERIAL AT L.M. POS. DETECTOR
532: BP STORAGE CONV, CYCLE BUSY	
113: BREAKER 2, APPLY CYCLE BUSY
968: BEADLOADER, SAFETY SCREEN AT FRONT ACTIVATED
220: SAFETY SCREEN (FLOOR) TBM B&T SIDE ACTIVE
184: CAPSTRIP LET-OFF, FESTOONER NOT AT OPERATING POS
999: BT SIDE- AREA_ENTRY_ACTIVED
870: CARCASS SIDE, SAFETY ACTIVATED
692: PREASSEMBLY LENGTH NOT CORRECT
697: PREASSEMBLY WIDTH NOT CORRECT
269: BT DRUM, MODULE DISABLED
524: BP PRODUCTION, CYCLE BUSY

